# To-do for first day of class

- Review Reddit code from Text Mining, implement it here. 
- Add more candidates, think about what fields we need
- Take all Twitter followers and their descriptions from candidates (this will take a fucking LONG time)
- Pull Twitter followers that don't follow the other candidates, or have candidate hashtags in descriptions
- take random sample. Think about random sample. e.g. Can we pull most active Tweeters? By Location?
- Look into Facebook API, try to implement it for candidates pages
- Make code only have to run once (Ideally every Sunday for the last week's data)
- Put data in GBQ
- Connect Tableau to GBQ